# Predicting Zomato Restaurant Rate

### 1. Business Problem

### 1.1 Problem Description

- Does demography of area matters?
- Does location of particular type of restraurant depends on people living in that area>
- Does theme of restraurant matters?
- Is food chain category restraurant likely to have more customers than its counter part?
- Are any neighbourhood on similar based on the type of food?
- Is particular neighbours is famous for itw own kind of food?
- If two neighbours are similar does that mean these are related or particular group of people live in neighbourhood or these are places to eat.
- What kind of food is famous in locality.
- Do entire locality loves veg food, if yes then locality populated by particular set of people eg Jain, Gujarati,Marwadi who are basically veg.


### 1.2. Problem Statement
The dataset also contains reviews for each of the restaurant which will help in finding overall rating for the place. So we will try to predict rating for particular restaurant.

### 1.3 Business Objectives

### 1.4 Machine Learning  Formulation

### 1.5 Performance Metrics

## 2. Data

### 2.1 Data Download

Zomato API was used to download data. Data was fetched from Zomato API by making zomatoWrapper package. Description on zomatoWrapper package can be found here:

https://github.com/ashish-gh/zomatoWrapper

Zomato API provides data in json which was later stored in .csv file. Particularly two API endpoints were used to fetch data. Finally the data were merged and final csv file was produced. Data storage manipulation process can be found here:

https://github.com/ashish-gh/Exploratory_Data_Analysis_Zomato_Restaurant/blob/master/DataStorage_and_Pre-processing.ipynb


### 2.2 Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

from wordcloud import WordCloud, STOPWORDS
from sklearn.preprocessing import OneHotEncoder

from joblib import dump,load
#%matplotlib notebook
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import  make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import random
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor